In [55]:
# Importamos las librerias necesarias 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
from wordcloud import WordCloud

LEER EL ARCHIVO CON LOS DATOS LIMPIOS

In [56]:
# Carga el primer archivo CSV
Datos_iniciales = pd.read_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datosfinal_google.csv')

Datos_iniciales.head(2)

,user_id,time,rating,text,gmap_id,year,restaurant_name,address,latitude,longitude,category,avg_rating,num_of_reviews,city
0,105941553219163737304,2018-09-20 00:12:47,1,"Love this place. Great little restaurant, with...",0x8894b5a7a7909725:0xe4687299fd288188,2018,gormley's on the river | modern cuisine in his...,gormley's on the river | modern cuisine in his...,29.726949,-84.981535,Modern French restaurant,4.3,17,apalachicola
1,105941553219163737304,2018-09-20 00:12:47,1,"Love this place. Great little restaurant, with...",0x8894b5a7a7909725:0xe4687299fd288188,2018,gormley's on the river | modern cuisine in his...,gormley's on the river | modern cuisine in his...,29.726949,-84.981535,Restaurant,4.3,17,apalachicola


De acuerdo a los datos realizamos un analisis de sentimiento donde vamos a obtener un valor Positivo, negativo o neutro de acuerdo al servicio 

In [57]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Descargar los recursos necesarios de NLTK (si no están descargados)
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Cargar el archivo CSV
archivo = 'c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datosfinal_google.csv'
data = pd.read_csv(archivo)

# Verifica si la columna 'text' existe en el DataFrame
if 'text' not in data.columns:
    raise ValueError("La columna 'text' no se encuentra en el archivo CSV")

# Preprocesar los textos (tokenización, minúsculas, eliminar stopwords)
def preprocess_text(text):
    tokens = word_tokenize(str(text).lower())  # Convertir a minúsculas y tokenizar
    stop_words = set(stopwords.words('english'))  # Stopwords en inglés
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Filtrar palabras
    return " ".join(tokens)

data['clean_text'] = data['text'].apply(preprocess_text)

# Análisis de sentimientos con VADER
sid = SentimentIntensityAnalyzer()
data['sentiment_score'] = data['clean_text'].apply(lambda x: sid.polarity_scores(x)['compound'])

# Etiquetar el sentimiento
def label_sentiment(score):
    if score > 0.05:
        return 'Positivo'
    elif score < -0.05:
        return 'Negativo'
    else:
        return 'Neutro'

data['sentiment'] = data['sentiment_score'].apply(label_sentiment)

# Verifica si la columna 'rating' existe para calcular el promedio
if 'rating' not in data.columns:
    raise ValueError("La columna 'rating' no se encuentra en el archivo CSV")

# Calcular el promedio de rating por restaurante
avg_rating = data.groupby('restaurant_name')['rating'].mean().reset_index()
avg_rating.rename(columns={'rating': 'average_rating'}, inplace=True)

# Unir el promedio de rating al dataset original
data = pd.merge(data, avg_rating, on='restaurant_name', how='left')

# Seleccionar las columnas solicitadas
columns = [
    'user_id', 'text', 'gmap_id', 'restaurant_name', 'address', 'latitude',
    'longitude', 'category', 'average_rating', 'num_of_reviews', 'city',
    'sentiment_score', 'sentiment'
]
final_data = data[columns]

# Mostrar la tabla final
print(final_data.head())

# Guardar la tabla en un archivo CSV (opcional)
resultado_archivo = 'c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/sentiment_analysis_results.csv'
final_data.to_csv(resultado_archivo, index=False)
print(f"Los resultados del análisis de sentimiento se guardaron en: {resultado_archivo}")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                 user_id                                               text  \
0  105941553219163737304  Love this place. Great little restaurant, with...   
1  105941553219163737304  Love this place. Great little restaurant, with...   
2  109857700811373231502  A classy restaurant to match the charm of the ...   
3  109857700811373231502  A classy restaurant to match the charm of the ...   
4  101770467939555366215  Highly recommended; great spot for locals and ...   

                                 gmap_id  \
0  0x8894b5a7a7909725:0xe4687299fd288188   
1  0x8894b5a7a7909725:0xe4687299fd288188   
2  0x8894b5a7a7909725:0xe4687299fd288188   
3  0x8894b5a7a7909725:0xe4687299fd288188   
4   0x88d900575f0dd065:0x9b3638d2a80be4d   

                                     restaurant_name  \
0  gormley's on the river | modern cuisine in his...   
1  gormley's on the river | modern cuisine in his...   
2  gormley's on the river | modern cuisine in his...   
3  gormley's on the river | modern c

Exportamos a un nuevo archivo los resultados

In [58]:
# Lee el archivo CSV
df = pd.read_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/sentiment_analysis_results.csv')


In [59]:
df.head(2)

,user_id,text,gmap_id,restaurant_name,address,latitude,longitude,category,average_rating,num_of_reviews,city,sentiment_score,sentiment
0,105941553219163737304,"Love this place. Great little restaurant, with...",0x8894b5a7a7909725:0xe4687299fd288188,gormley's on the river | modern cuisine in his...,gormley's on the river | modern cuisine in his...,29.726949,-84.981535,Modern French restaurant,3.0,17,apalachicola,0.9186,Positivo
1,105941553219163737304,"Love this place. Great little restaurant, with...",0x8894b5a7a7909725:0xe4687299fd288188,gormley's on the river | modern cuisine in his...,gormley's on the river | modern cuisine in his...,29.726949,-84.981535,Restaurant,3.0,17,apalachicola,0.9186,Positivo


El archivo anterior tiene ciudades vacias, es decir no se han logrado extraer de acuerdo con la direccion. Entonces vamos a Asignarle una ciudad de acuerdo al codigo postal. 
Extraemos un archivo de codigos postales de Florida tomado de la web 

In [60]:
# Leer el archivo CSV
codigos_posteles = pd.read_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/Codigos_postales.csv')

codigos_posteles.head()

,ZIP CODE,STATE,CITY,SERVICE DAY STANDARD,TERMINAL,SCAC Code,RELATED SURCHARGES,FLAT AMT,RATE,MIN AMT,SUN,MON,TUE,WED,THU,FRI,SAT
0,32615,FL,ALACHUA,0,MEMP,SEFL,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32616,FL,ALACHUA,0,MEMP,SEFL,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32420,FL,ALFORD,0,MEMP,SEFL,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32701,FL,ALTAMONTE SPG,0,MEMP,SEFL,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32714,FL,ALTAMONTE SPG,0,MEMP,SEFL,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creamos un diccionario que contecta los codigos postales y las ciudades correspondientes

In [61]:

# Leer el archivo CSV con los códigos postales
path_to_zipcodes = 'c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/Codigos_postales.csv'
codigos_postales = pd.read_csv(path_to_zipcodes)

# Crear el diccionario de códigos postales y ciudades
codigo_postal_ciudades = dict(zip(codigos_postales['ZIP CODE'].astype(str), codigos_postales['CITY']))

# Verificar el resultado
print("Diccionario creado con éxito. Ejemplo:")
print({k: codigo_postal_ciudades[k] for k in list(codigo_postal_ciudades.keys())[:5]})  # Muestra los primeros 5


Diccionario creado con éxito. Ejemplo:
{'32615': 'ALACHUA', '32616': 'ALACHUA', '32420': 'ALFORD', '32701': 'ALTAMONTE SPG', '32714': 'ALTAMONTE SPG'}


In [62]:
import pandas as pd
import re  # Para trabajar con expresiones regulares

# Leer el archivo con los datos
df = pd.read_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/sentiment_analysis_results.csv')

# Función para extraer el código postal desde la columna `address`
def extraer_codigo_postal(address):
    try:
        # Intentar extraer código postal precedido por "邮政编码:"
        match = re.search(r'邮政编码:\s*(\d{5})', str(address))
        if match:
            return match.group(1)
        # Si no, buscar un código postal estándar (5 dígitos) al final de la dirección
        match = re.search(r'(\d{5})(?!\d)', str(address))
        if match:
            return match.group(1)
    except:
        pass
    return None  # Devolver None si no se encuentra un código postal

# Crear una nueva columna con los códigos postales
df['postal_code'] = df['address'].apply(extraer_codigo_postal)

# Guardar el DataFrame actualizado para revisar los resultados
df.to_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_con_codigos_postales.csv', index=False)

print("Los códigos postales han sido extraídos y guardados en una nueva columna.")



Los códigos postales han sido extraídos y guardados en una nueva columna.


In [63]:
# Leer el archivo con los datos
datosfinales_con_zip = pd.read_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_con_codigos_postales.csv')

datosfinales_con_zip.head()

,user_id,text,gmap_id,restaurant_name,address,latitude,longitude,category,average_rating,num_of_reviews,city,sentiment_score,sentiment,postal_code
0,105941553219163737304,"Love this place. Great little restaurant, with...",0x8894b5a7a7909725:0xe4687299fd288188,gormley's on the river | modern cuisine in his...,gormley's on the river | modern cuisine in his...,29.726949,-84.981535,Modern French restaurant,3.0,17,apalachicola,0.9186,Positivo,32320
1,105941553219163737304,"Love this place. Great little restaurant, with...",0x8894b5a7a7909725:0xe4687299fd288188,gormley's on the river | modern cuisine in his...,gormley's on the river | modern cuisine in his...,29.726949,-84.981535,Restaurant,3.0,17,apalachicola,0.9186,Positivo,32320
2,109857700811373231502,A classy restaurant to match the charm of the ...,0x8894b5a7a7909725:0xe4687299fd288188,gormley's on the river | modern cuisine in his...,gormley's on the river | modern cuisine in his...,29.726949,-84.981535,Modern French restaurant,3.0,17,apalachicola,0.6808,Positivo,32320
3,109857700811373231502,A classy restaurant to match the charm of the ...,0x8894b5a7a7909725:0xe4687299fd288188,gormley's on the river | modern cuisine in his...,gormley's on the river | modern cuisine in his...,29.726949,-84.981535,Restaurant,3.0,17,apalachicola,0.6808,Positivo,32320
4,101770467939555366215,Highly recommended; great spot for locals and ...,0x88d900575f0dd065:0x9b3638d2a80be4d,zampini's bottega,"zampini's bottega, 510 se 5th ave, fort lauder...",26.116549,-80.138561,Restaurant,5.0,18,fort lauderdale,0.9466,Positivo,33301


In [65]:
import pandas as pd

# Leer el archivo CSV con los datos
datosfinales_con_zip = pd.read_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_con_codigos_postales.csv')

# Guardar el DataFrame como un archivo Excel
datosfinales_con_zip.to_excel('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_con_codigos_postales.xlsx', index=False)

print("El archivo ha sido convertido a Excel y guardado correctamente.")


El archivo ha sido convertido a Excel y guardado correctamente.


In [53]:
import pandas as pd

# Leer el archivo con los datos
df = pd.read_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_con_codigos_postales.csv')

# Diccionario de códigos postales y ciudades
diccionario_codigos_postales = {
    '32320': 'Apalachicola',
    '33301': 'Fort Lauderdale',
    # Añade aquí más códigos postales con sus ciudades
}

# Función para asignar la ciudad correspondiente
def asignar_ciudad(postal_code):
    return diccionario_codigos_postales.get(str(postal_code), 'Desconocido')

# Actualizar la columna 'city' usando el diccionario
df['city'] = df['postal_code'].apply(asignar_ciudad)

# Guardar el DataFrame actualizado para revisar los resultados
df.to_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_con_codigos_postales_actualizados.csv', index=False)

print("La columna 'city' ha sido actualizada con las ciudades correspondientes.")


La columna 'city' ha sido actualizada con las ciudades correspondientes.


In [69]:
import pandas as pd

# Leer el archivo CSV con los datos principales
path_to_data = 'c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_con_codigos_postales.csv'
df = pd.read_csv(path_to_data)

# Leer el archivo CSV con los códigos postales
path_to_zipcodes = 'c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/Codigos_postales.csv'
codigos_postales = pd.read_csv(path_to_zipcodes)

# Crear el diccionario de códigos postales y ciudades
codigo_postal_ciudades = dict(zip(codigos_postales['ZIP CODE'].astype(str), codigos_postales['CITY']))

# Verificar que el diccionario se creó correctamente (opcional)
print("Diccionario creado con éxito. Ejemplo:")
print({k: codigo_postal_ciudades[k] for k in list(codigo_postal_ciudades.keys())[:5]})  # Muestra los primeros 5

# Función para asignar la ciudad correspondiente
def asignar_ciudad(postal_code):
    return codigo_postal_ciudades.get(str(postal_code), 'Desconocido')  # Usa 'Desconocido' si el código no está en el diccionario

# Actualizar la columna 'city' usando el diccionario
df['city'] = df['postal_code'].apply(asignar_ciudad)

# Guardar el DataFrame actualizado en un nuevo archivo CSV
output_path = 'c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_con_codigos_postales_actualizados.csv'
df.to_csv(output_path, index=False)

print(f"La columna 'city' ha sido actualizada con las ciudades correspondientes. Archivo guardado en {output_path}.")


Diccionario creado con éxito. Ejemplo:
{'32615': 'ALACHUA', '32616': 'ALACHUA', '32420': 'ALFORD', '32701': 'ALTAMONTE SPG', '32714': 'ALTAMONTE SPG'}
La columna 'city' ha sido actualizada con las ciudades correspondientes. Archivo guardado en c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_con_codigos_postales_actualizados.csv.


In [71]:
df = df[df['city'] != 'Desconocido']

df = df[df['city'].notna() & (df['city'] != 'Desconocido')]


In [72]:
# Guardar el DataFrame como un archivo CSV
df.to_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_filtrados.csv', index=False)


In [73]:
import pandas as pd

# Leer el archivo CSV con los datos filtrados
df = pd.read_csv('c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_filtrados.csv')

# Guardar el DataFrame como un archivo Excel
output_path_excel = 'c:/DataScience/proyect-google-yelp/ETL_EDA_GOOGLE/datos_filtrados.xlsx'
df.to_excel(output_path_excel, index=False)

print("El DataFrame ha sido guardado correctamente en un archivo Excel.")


El DataFrame ha sido guardado correctamente en un archivo Excel.
